In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline
import os

def clean_price(price):
    """Clean and convert Tunisian price strings to floats"""
    if isinstance(price, str):
        price = price.replace('DT', '').replace('\xa0', '').replace(',', '').strip()
        try:
            return float(price)
        except ValueError:
            return np.nan
    return price

def load_and_preprocess_data(filepath):
    """Load and preprocess the pricing data"""
    try:
        df = pd.read_csv(filepath)
        
        # Clean price columns
        price_columns = ['price', 'historical_price', 'price_tunisianet', 'price_mytech']
        for col in price_columns:
            if col in df.columns:
                df[col] = df[col].apply(clean_price)
        
        # Proper way to fill NA values without chained assignment
        numerical_cols = price_columns + ['historical_discount']
        for col in numerical_cols:
            if col in df.columns:
                # Recommended alternative to inplace=True
                df[col] = df[col].fillna(df[col].median())
        
        return df
    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return None

def feature_engineering(df):
    """Create additional features for the model"""
    df = df.copy()  # Ensure we're working on a copy
    
    # Competitor price features
    df['price_diff_competitors'] = df['price_tunisianet'] - df['price_mytech']
    df['price_ratio_competitors'] = df['price_tunisianet'] / df['price_mytech']
    df['discount_impact'] = df['historical_discount'] * df['historical_price']
    
    # Encode categorical variables
    le = LabelEncoder()
    df['category_encoded'] = le.fit_transform(df['category'])
    
    return df

def train_model(df):
    """Train the price prediction model"""
    features = ['historical_price', 'price_tunisianet', 'price_mytech', 
               'historical_discount', 'price_diff_competitors',
               'price_ratio_competitors', 'discount_impact', 'category_encoded']
    target = 'price'
    
    X = df[features]
    y = df[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Evaluate model
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f"Model Mean Absolute Error: {mae:.2f}")
    
    return model

def format_tnd(price):
    """Format price in Tunisian Dinar format"""
    return f"{price:,.0f} DT".replace(',', '\xa0')

def generate_results(df, model):
    """Generate and format the final results"""
    features = ['historical_price', 'price_tunisianet', 'price_mytech', 
               'historical_discount', 'price_diff_competitors',
               'price_ratio_competitors', 'discount_impact', 'category_encoded']
    # Make predictions
    df['predicted_optimal_price'] = model.predict( df[features])
    
    # Apply business rules
    df['recommended_price'] = np.where(
        df['predicted_optimal_price'] < df['historical_price'] * 0.9,
        df['historical_price'] * 0.9,
        df['predicted_optimal_price']
    )
    
    # Create formatted columns
    for col in ['historical_price', 'price_tunisianet', 'price_mytech', 
               'predicted_optimal_price', 'recommended_price']:
        df[f"{col}_formatted"] = df[col].apply(format_tnd)
    
    # Calculate price increase percentage
    df['price_increase_pct'] = ((df['recommended_price'] - df['historical_price']) / 
                               df['historical_price'] * 100)
    
    # Prepare final results
    results = df[['id', 'title', 'category', 'historical_price_formatted', 
                 'price_tunisianet_formatted', 'price_mytech_formatted', 
                 'predicted_optimal_price_formatted', 'recommended_price_formatted',
                 'price_increase_pct']].copy()
    
    return results

# Main execution
if __name__ == "__main__":
    # Replace with your actual file path
    input_path = r"C:\Users\medan\Desktop\modelsai\scraped_data12.csv"
    output_path = r'C:\Users\medan\Desktop\modelsai\price_recommendations.csv'
    
    # Load and preprocess data
    df = load_and_preprocess_data(input_path)
    if df is not None:
        # Feature engineering
        df = feature_engineering(df)
        
        # Train model
        model = train_model(df)
        
        # Generate results
        results = generate_results(df, model)
        
        # Save results
        results.to_csv(output_path, index=False)
        print(f"Results saved to {output_path}")
        
        # Show top recommendations
        print("\nTop 10 Products with Highest Recommended Price Increases:")
        print(results.sort_values('price_increase_pct', ascending=False).head(10))

Model Mean Absolute Error: 4381797.50
Results saved to C:\Users\medan\Desktop\modelsai\price_recommendations.csv

Top 10 Products with Highest Recommended Price Increases:
    id                                              title     category  \
7   82          Playstation Portal PS5 Remote Player Sony  electronics   
11  86  Setup Gamer Special Pc de Bureau Gaming / Ryze...  electronics   
2   77                Disque backup HDD plus hub-16T  3.0  electronics   
1   76  Tablette Logicom La Tab 74 / 7" / Wifi / 3G / ...  electronics   
9   84  Pc Portable Lenovo ThinkPad L16 Gen 1 / Ultra ...  electronics   
0   75             ECOUTEUR Sans Fil JBL Tour Pro 3 BEIGE  electronics   
12  87  Setup Gamer Special Pc de Bureau Gaming / i5-1...  electronics   
8   83  Pc de bureau Gamer ELITE / Ryzen 9 7950X / RTX...  electronics   
13  88  Studio Monitor 4 Hi-Fi Energy Sistem Avec Télé...  electronics   
3   78     Console de jeu Sony PlayStation 5 Slim Digital  electronics   

   historical